# Baysian Networks
with R kernel


Bnlearn Tutorial: https://jacintoarias.github.io/bayesnetRtutorial/#the_bnlearn_package

In [1]:
list_of_packages <- c ("synthpop", "insight", "party", "dplyr", "rpart", "rpart.plot", "randomForest", "pROC", "caret", "pracma", "bnlearn", "arulesCBA")

lapply(list_of_packages, FUN= function(X){
  do.call("require", list(X))
})

Lade n"otiges Paket: synthpop

Find out more at https://www.synthpop.org.uk/

Lade n"otiges Paket: insight

Lade n"otiges Paket: party

Lade n"otiges Paket: grid

Lade n"otiges Paket: mvtnorm

Lade n"otiges Paket: modeltools

Lade n"otiges Paket: stats4

Lade n"otiges Paket: strucchange

Lade n"otiges Paket: zoo


Attache Paket: 'zoo'


Die folgenden Objekte sind maskiert von 'package:base':

    as.Date, as.Date.numeric


Lade n"otiges Paket: sandwich

Lade n"otiges Paket: dplyr


Attache Paket: 'dplyr'


Das folgende Objekt ist maskiert 'package:party':

    where


Die folgenden Objekte sind maskiert von 'package:stats':

    filter, lag


Die folgenden Objekte sind maskiert von 'package:base':

    intersect, setdiff, setequal, union


Lade n"otiges Paket: rpart

Lade n"otiges Paket: rpart.plot

Lade n"otiges Paket: randomForest

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attache Paket: 'randomForest'


Das folgende Objekt ist maskiert 'package:dpl

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

# Data

In [2]:
directory <- "/Users/emmafoessing/Documents/Master/MA/Code/Master-Thesis"

In [3]:
load(paste0(directory,"/cpspop.RData"))
adult <- read.csv(paste0(directory, "/adult_preprocessed.csv"))

In [4]:

# delete NAs
adult[adult == "?"] <- NA
adult <- na.omit(adult)

adult$workclass <- as.factor(adult$workclass)
adult$education <- as.factor(adult$education)
adult$marital_status <- as.factor(adult$marital_status)
adult$relationship <- as.factor(adult$relationship)
adult$race <- as.factor(adult$race)
adult$sex <- as.factor(adult$sex)
adult$native_country <- as.factor(adult$native_country)
adult$income <- as.factor(adult$income)

adult_with_cont <- adult
cps_with_cont <- cpspop

Ich diskretisiere alle meine continuous cars (in Intervalle einteilen) --> das geht mit der discretize function <br>
Ich will ca. 5 Kategorien maximal pro Variable haben. Wenn eine der Ausprägungen in der Varibale mindestens 1/5 aller Ausprägungnen aus macht, dann soll diese eine eigene Kategorie werden und die restlichen als Interval kategorisiert werden. Wenn es mehrere Werte gibt, die mindestens 1/5 aller Ausprägungen ausmachen, dann sollen auch diese alle jeweils eine eigene Kategorie werden und der Rest kann in Intervalle eingeteilt werden. Am Ende soll die Variable des Datensatzes überschrieben werden mit der kategorialen Variable und der Datensatz nur aus 'factor' Variablen bestehen.
2/5 sind =0 --> 3 weitere Kategorien mit Intervallen
3/5 sind =0 --> 2 weitere Kategorien mit Intervallen

In [5]:
discretize_df = function(df, breaks = 5) {
  for (var in colnames(df)) {
    # Check if the variable is not a factor
    if (!is.factor(df[[var]])) {

      # Count the frequency of each unique value
      freq_table <- table(df[[var]])

      # Calculate the proportion of zeros, ensuring NA is handled
      zero_proportion <- ifelse(!is.na(freq_table[as.character(0)]), 
                                freq_table[as.character(0)] / sum(freq_table), 
                                0)

      # Determine the number of breaks based on zero proportion
      if (zero_proportion > 4/5) {
        new_breaks = 1
      } else if (zero_proportion > 1/4) {
        new_breaks = breaks - 2
      } else if (zero_proportion > 1/5) {
        new_breaks = breaks - 1
      } else {
        new_breaks = breaks
      }
      
      # Separate zeros and non-zeros
      zero_portion = (df[[var]] == 0)
      non_zero_values = df[[var]][!zero_portion]

      # Discretize non-zero values
      if (length(non_zero_values) > 0) {
        # Calculate breaks for non-zero values
        range_values = range(non_zero_values, na.rm = TRUE)
        breaks_values = seq(range_values[1], range_values[2], length.out = new_breaks + 1)
        
        # Ensure correct number of labels are created
        labels = sapply(1:(length(breaks_values)-1), function(i) 
                        paste("(", breaks_values[i], "-", breaks_values[i+1], "]", sep=""))

        # Use cut to apply these breaks and labels
        discretized_non_zeros = cut(non_zero_values, breaks = breaks_values, labels = labels, include.lowest = TRUE)
        # Combine zero and discretized non-zeros into the original dataframe
        df[[var]] <- factor(ifelse(zero_portion, "0", as.character(discretized_non_zeros)))
      } else {
        # If all values are zero or the number of breaks is zero or negative
        df[[var]] <- factor("0")
      }
    }
  }
  return(df)
}

In [6]:
cps_dis <- discretize_df(cpspop)
levels(cps_dis$csp)

[1] "(1-23917]" "0"

In [7]:
# Using a Baysian Network to generate synthetic data
baysian_net <- function(data){
    # discretize data before creating syn. data
    #data <- discretize_df(data)
    bn <- hc(data)  # Learn structure using Hill-Climbing
    bn <- bn.fit(bn, data)  # Learn parameters using Maximum Likelihood Estimation

    # Generate Synthetic Data
    syn_data <- rbn(bn, length(data))  # Generate same number of obs as in the original data

    return(syn_data)
    }

# Model

Problem: Baysian networks require data structure with solely discrete variables!
--> exists in both CPS and Adult
For continuous data, common practice would be to first discretize the variables (convert the continuous data into bins or categories) and then apply Bayesian Network algorithms. However, the process of discretization must be done very thoughtfully as ill-advised discretization can lead to loss of crucial information.

In [8]:
adult_syn <- baysian_net(discretize_df(adult))

In [9]:
cps_syn <- baysian_net(discretize_df(cps_with_cont))

In [10]:
head(cps_syn)

,tax,income,csp,age,educ,marital,race,sex,ss
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,(1-33333],(1-153749.2],0,(30-45],39,1,1,1,0
2,(1-33333],(1-153749.2],0,(60-75],46,1,1,1,(7-16671.3333333333]
3,(1-33333],(1-153749.2],0,(45-60],39,1,1,1,0
4,(1-33333],(1-153749.2],0,(60-75],39,4,1,2,(7-16671.3333333333]
5,(1-33333],(1-153749.2],0,(30-45],40,1,1,1,0
6,(1-33333],(1-153749.2],0,(60-75],39,1,1,1,0


für die Simulation:
Predictions mit dem kategorisiertem Datensatz? Ergibt nur Sinn weil sonst sind die Bedingungen ganz verschiedene.
Aber: Wenn ich BN auch mit reinnehme als Modelle für die Predictions bei den anderen SDGs dann hat die Performance es schlechter --> überhaupt vergleichbar mit anderen Modellen? (letztendlich muss nur synthetisch gegen original verglichen werden)

## Simulation

### Helper functions

In [ ]:
# Define a function to perform the structure learning and fitting
train_bn <- function(data, algorithm, score) {
  if (algorithm == "hc") {
    bn_structure <- hc(data, score = score)
  } else if (algorithm == "tabu") {
    bn_structure <- tabu(data, score = score)
  } else if (algorithm == "gs") {
    bn_structure <- gs(data)
  } else if (algorithm == "iamb") {
    bn_structure <- iamb(data)
  }
  
  bn_fitted <- bn.fit(bn_structure, data)
  return(bn_fitted)
}

In [ ]:
# Define a function to evaluate the model using cross-validation
evaluate_bn <- function(data, bn_fitted, target_var) {
  predictions <- predict(bn_fitted, data = data, node = target_var)
  actual <- data[[target_var]]
  accuracy <- mean(predictions == actual)
  return(accuracy)
}

In [ ]:
# Calculate evaluation metrics for factored targets
evaluation_metrics_factor <- function(predictions, test_set){
    # confusion matrix for the prediction on original data
    cm <- confusionMatrix(predictions, test_set$income,
                mode = "everything")

    # saving evaluation metrics
    accuracy <- cm$overall['Accuracy']
    f1 <- cm$byClass['F1']
    sens <- cm$byClass['Sensitivity']
    spec <- cm$byClass['Specificity']

    # Create the dataframe
    metrics_df <- data.frame(
        Accuracy = accuracy, 
        F1 = f1, 
        Sensitivity = sens, 
        Specificity = spec
    )
    
    return(metrics_df)
}

In [ ]:
bn_simulation <- function(data, nrun = 10, kfold = 10, algorithms = c("hc", "tabu", "gs", "iamb"), scores = c("aic", "bic", "k2")) {

    # create empty list to store evaluation dataframes
    eval_list <- list()

    # discretize the data
    data <- discretize_df(data)

    # set inital seed
    s <- 1234
    for (i in 1:nrun){
        # vary seed with each run
        s <- s + 1
        set.seed(s)

        # generate synthetic data
        gen_data <- baysian_net(data)

        # Split the data into training and testing sets
        trainIndex <- createDataPartition(data$income, p = .8, 
                                            list = FALSE, 
                                            times = 1)
        trainData <- data[ trainIndex,]
        testData  <- data[-trainIndex,]
        
        # define control for CV
        control <- trainControl(method = "cv", number = kfold)

        train_control <- trainControl(method = "cv", number = 5)

        # Define the grid of parameters
        grid <- expand.grid(algorithm = algorithms, score = scores)
        
        # Define the target variable
        target_var <- "income"

        # Perform grid search with cross-validation
        results <- lapply(1:nrow(grid), function(i) {
            algorithm <- grid$algorithm[i]
            score <- ifelse(grid$algorithm[i] %in% c("hc", "tabu"), grid$score[i], NA)
            
            bn_fitted <- train_bn(data, algorithm, score)
            accuracy <- evaluate_bn(data, bn_fitted, target_var)
            
            return(data.frame(algorithm = algorithm, score = score, accuracy = accuracy))
        })

        # Combine results into a single data frame
        results <- do.call(rbind, results)

        # Select the best model
        best_model <- results[which.max(results$accuracy),]

        # Train the best model on the entire dataset
        final_bn <- train_bn(trainData, best_model$algorithm, best_model$score)
        
        # Use the final model for prediction
        predictions <- predict(final_bn, data = testData, node = target_var)

        # Evaluate the final model
        eval <- as.data.frame(evaluation_metrics_factor(predictions, testData))

        eval_list[[i]] <- eval
        print(c("run", i, "completed"))
    }

    # outside the nruns loop
    # average over all runs
    sum_df <- Reduce(function(x, y) Map(`+`, x, y), eval_list)
    eval_avg <- lapply(sum_df, function(col) col / length(eval_list))

    # Convert the list back to a dataframe
    # Store row names
    rownames <- row.names(eval_list[[1]])

    # Convert the list back to a dataframe
    eval_avg <- as.data.frame(eval_avg)

    # Set back the row names
    rownames(eval_avg) <- row.names(eval_list[[1]])

    # Return the average evaluation metrics
    return(eval_avg)
}